We activate the specific "TimeIndependentSchrodingerEquation" environment

In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/github_repositories/my_repositories/TimeIndependentSchrodingerEquation`


We need to execute the following commands if we build the environment for the first time

In [ ]:
#= Pkg.add("Gridap")
Pkg.add("GridapGmsh")
Pkg.add("Gmsh");
Pkg.add("FileIO");
Pkg.add("LinearAlgebra");
Pkg.add("SparseArrays");
Pkg.add("Arpack"); =#

With this command we can check the intalled packages in the environment

In [19]:
Pkg.status()

Project TimeIndependentSchrodingerEquation v1.0.0-DEV
Status `~/github_repositories/my_repositories/TimeIndependentSchrodingerEquation/Project.toml`
  [7d9fca2a] Arpack v0.5.4
  [5789e2e9] FileIO v1.16.1
  [705231aa] Gmsh v0.2.2
  [56d4f2e9] Gridap v0.17.17
  [3025c34a] GridapGmsh v0.6.1
  [37e2e46d] LinearAlgebra
  [44cfe95a] Pkg v1.9.0
  [2f01184e] SparseArrays
Warning The project dependencies or compat requirements have changed since the manifest was last resolved. It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.


In [20]:
Pkg.resolve()

  No Changes to `~/github_repositories/my_repositories/TimeIndependentSchrodingerEquation/Project.toml`
  No Changes to `~/github_repositories/my_repositories/TimeIndependentSchrodingerEquation/Manifest.toml`


In [ ]:
Pkg.instantiate()

In [21]:
using TimeIndependentSchrodingerEquation
using Gridap,Test

### Check functions inside `MeshGeneratorFunction.jl`

In [3]:
dom2D=(-25.0,25.0,-25.0,25.0);nxy=(50,50);params_model=((dom2D,nxy));
grid_type="Cartesian2D";

In [4]:
model2D=make_model(grid_type,params_model);

In [5]:
#= using Gridap
writevtk(model2d,"model2d") =#

### Check functions inside `BoundaryConditionsFunction.jl`

In [6]:
BC_type="FullDirichlet";
FullDirichlet_values,FullDirichlet_tags=make_boundary_conditions(grid_type,BC_type,ComplexF64);

### Check functions inside `MiscellaneousFunctions.jl`

In [7]:
Ω,dΩ,Γ,dΓ = measures(model2D,3,FullDirichlet_tags)

(BodyFittedTriangulation(), GenericMeasure(), BoundaryTriangulation(), GenericMeasure())

In [8]:
reff = ReferenceFE(lagrangian,Float64,2)

(Lagrangian(), (Float64, 2), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [23]:
VSpace,USpace = fe_spaces(model2D,reff,grid_type;BC_type=BC_type,TypeData=ComplexF64)

(UnconstrainedFESpace(), TrialFESpace())

#### We build functions to write a specific Sturm-Liouville formulation

In [24]:
function eigenvalue_problem_functions(params;switch_potential = "QHO_1D")
    ħ::Float64=1.0; m::Float64=1.0;
    if (switch_potential == "QHO_1D")
        # 1D quantum harmonic oscillator potential
        println("Set quantum harmonic oscillator 1D potential");
        ω,x₁=params;
        p_QHO_1D(x) = 0.5*(ħ*ħ)*(1.0/m);                                      # kinetic energy
        q_QHO_1D(x) = 0.5*m*(ω*ω)*(x[1]-x₁)*(x[1]-x₁);                        # 1D harmonic oscillator centered in x₁
        r_QHO_1D(x) = 1.0;
        return p_QHO_1D,q_QHO_1D,r_QHO_1D
    elseif (switch_potential == "QHO_2D")
        # 2D quantum harmonic oscillator potential
        println("Set quantum harmonic oscillator 2D potential");
        ω,x₁,y₁=params;
        p_QHO_2D(x) = 0.5*(ħ*ħ)*(1.0/m);                                       # kinetic energy
        q_QHO_2D(x) = 0.5*m*(ω*ω)*((x[1]-x₁)*(x[1]-x₁)+(x[2]-y₁)*(x[2]-y₁));   # 2D harmonic osicllator centered in (x₁,y₁)
        r_QHO_2D(x) = 1.0;
        return p_QHO_2D,q_QHO_2D,r_QHO_2D
    end
end

eigenvalue_problem_functions (generic function with 1 method)

In [25]:
p,q,r = eigenvalue_problem_functions((1.0,0.0,0.0);switch_potential = "QHO_2D")

Set quantum harmonic oscillator 2D potential


(var"#p_QHO_2D#22"{Float64, Float64}(1.0, 1.0), var"#q_QHO_2D#23"{Float64, Float64}(0.0, Core.Box(0.0), Core.Box(1.0), 1.0), var"#r_QHO_2D#24"())

### Check functions inside `EigenProblemSolveFunction.jl`

In [26]:
ϵ,ϕ = eigen_values_and_eigen_vectors(p,q,r,dΩ,USpace,VSpace;params=(4,10e-9,500,:none,0.0))

(ComplexF64[1.0045798611878358 - 1.7750358144878927e-17im, 2.017535772414723 - 4.9427743429198825e-17im, 2.017535772414729 + 1.0139307186121849e-16im, 3.030491683641623 - 8.535042376344906e-17im], CellField[SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction()])

#### We build an array with exactly eigenenergie values to quantum 2D harmonic oscillator

In [27]:
function exactly_eigenvalues_2dqho(num_eigval::Integer)
    ϵ_real_aux=Array{Float64}(undef, num_eigval^2)
    index::Int64=1
    for i in 1:num_eigval
        for j in 1:num_eigval
            ϵ_real_aux[index]=((i-1)+(j-1)+1)
            index+=1
        end
    end
    ϵ_real_aux=sort(ϵ_real_aux)
    ϵ_real = ϵ_real_aux[1:num_eigval];
    return ϵ_real
end

exactly_eigenvalues_2dqho (generic function with 1 method)

In [28]:
ϵ_real=exactly_eigenvalues_2dqho(length(ϵ))

4-element Vector{Float64}:
 1.0
 2.0
 2.0
 3.0

In [29]:
@test real(ϵ) ≈ ϵ_real atol=0.1

Test Passed

#### We define useful functions to check the eigenvectors

In [30]:
function norm_l2(𝜳::CellField,dΩ::Gridap.CellData.GenericMeasure)
    return sqrt(real(sum(∫(𝜳'*𝜳)*dΩ)));
end

norm_l2 (generic function with 1 method)

In [31]:
@testset "CheckNormalization" begin
    for i in eachindex(ϕ)
        @test norm_l2(ϕ[1],dΩ) ≈ 1.0
    end
end


Test Summary:      | Pass  Total  Time
CheckNormalization |    4      4  2.4s


Test.DefaultTestSet("CheckNormalization", Any[], 4, false, false, true, 1.684434345511512e9, 1.684434347910299e9, false)

In [32]:
function ortho_check_v2(ϕ::Vector{CellField},TrialSpace::FESpace,dΩ::Gridap.CellData.GenericMeasure)
    nev=length(ϕ)
    OrthoVector=zeros(Float64,round(Int,(nev^2-nev)/2));
    index::Int64=1
    for i in 2:nev
        ϕᵢ=interpolate_everywhere(ϕ[i],TrialSpace);
        for j in 1:(i-1)
            ϕⱼ=interpolate_everywhere(ϕ[j],TrialSpace);
            OrthoVector[index]=abs(sum(∫(ϕⱼ'*ϕᵢ)*dΩ))
            index+=1
        end
    end
    return OrthoVector;
end

ortho_check_v2 (generic function with 1 method)